In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

import logging
import re
import sys
import urllib.request
import time

from bs4 import BeautifulSoup
from queue import Queue
from urllib import parse, request
from queue import PriorityQueue
from operator import itemgetter
import re

urls = ["https://www.dailyscript.com/movie.html", "https://www.dailyscript.com/movie_n-z.html"]

def parse_links(root, html):
    soup = BeautifulSoup(html, 'html.parser')
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            text = link.string
            if not text:
                text = ''
            text = re.sub('\s+', ' ', text).strip()
            if "scripts" in link.get('href') and ".html" in link.get('href'):
                yield (parse.urljoin(root, link.get('href')))

def get_links(urls):
    l = []
    for url in urls:
        res = request.urlopen(url)
        l = l + list(parse_links(url, res.read()))
    return l

def get_titles(urls):
    t = []
    for url in urls:
        res = request.urlopen(url)
        soup = BeautifulSoup(res.read(), 'html.parser')
        for link in soup.find_all('a'):
            href = link.get('href')
            if href:
                text = link.string
                if not text:
                    text = ''
                text = re.sub('\s+', ' ', text).strip()
                if "scripts" in link.get('href') and ".html" in link.get('href'):
                    t.append((re.search(r'<a.*?>(.*?)</a>', str(link))).group(1))
    return t


def countOccurrence(a):
    k = {}
    not_in_list = ["I", "INT", "AND", "EXT", "ED", "HE", "ON", "OF", "THE", "CUT", "DR", "MRS", "IN", "TO", "MR", "CLOSE", "FADE", "LOC"]
    for j in a:
        pattern = r"[:.,]"
        j = re.sub(pattern, "", j)
        j = re.sub(r"[':]", "", j)
        j = re.sub(r"\s*\(.*\)", "", j)

        if j.isupper() and j not in not_in_list and len(j) > 1:
            if j in k:
                k[j] +=1
            else:
                k[j] = 1
    return dict(sorted(k.items(), key = itemgetter(1), reverse = True)[:5])

def get_html(url):
    res = request.urlopen(url)
    soup = str(BeautifulSoup(res, 'html.parser').text)
    splits = soup.split()
    counts = countOccurrence(splits)
    return counts, soup

def extract_dialogue(script, characters):
    dialogue = []
    match = False
    upper = False
    for line in script.split('\n'):
        match = False
        line = line.lstrip()
        for character in characters:
            if line.startswith(character):
                match = True
                upper = False
                speaker = character
        if match:
            spoken = line
            if speaker.isupper():
                if speaker.strip() != spoken.strip():
                    dialogue.append((speaker.strip(), spoken.strip()))
                else: 
                    dialogue.append((speaker.strip(), ""))
            # If the speaker isn't all uppercase, it's likely dialogue continuation or other text, so append it to the previous line
        elif len(dialogue) > 0:
            line_split = line.split()
            for l in line_split:
                if l.isupper():
                    upper = True
            if not upper:
                dialogue[-1] = (dialogue[-1][0], dialogue[-1][1] + ' ' + line.strip())
    return dialogue

def clean_text(dialogue_list, characters):
    returning_list = []
    for i in range(len(dialogue_list)):
        if len(dialogue_list[i][1]) == 0:
            continue
        if dialogue_list[i][0] not in characters:
            continue
        s = re.sub("[\(\[].*?[\)\]]", "", dialogue_list[i][1])
        s = " ".join(s.split())
        if len(s) != 0:
            returning_list.append((dialogue_list[i][0], s))
    return returning_list

def retrieve_in_string(cleaned_dialogue):
    s = ""
    for element in cleaned_dialogue:
        s += element[1] + "\n"
    return s

In [16]:
def generate_regex(movie_title):
    escaped_movie = re.escape(movie_title) + "\s*"
    regex = escaped_movie.replace("\ ", "\s+") + "(\([0-9a-zA-Z\\s]*\)\s*)*"
    return regex

In [17]:
def get_MBTI(url):
    driver = webdriver.Chrome()
    driver.get(url)
    personality = []
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'profile-name')))
    try:
        MBTI = driver.find_elements(By.CLASS_NAME, 'vote-detail-letter')
        for letter in MBTI:
            personality.append((letter.text)[-1])
        personality.append(((driver.find_element(By.CLASS_NAME, 'profile-description')).find_element(By.CLASS_NAME, 'container')).text)
    except:
        driver.quit()
        return personality
    driver.quit()
    return personality

In [18]:
def scrape_personalities(name, url):
    personality = ["", "", "", "", ""]
    driver = webdriver.Chrome()
    driver.get(url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'community-title')))
    try:
        characters = driver.find_elements(By.CLASS_NAME, 'profile-card-link')
        regex = generate_regex(name)
        for character in characters:
            a = character.find_element(By.CLASS_NAME, 'info-name')
            if (re.search(regex, a.text, re.IGNORECASE)):
                print("Found " + a.text)
                personality = get_MBTI(character.get_attribute('href'))
                print(personality + "\n")
                break
    except:
        driver.quit()
        return personality
    driver.quit()
    return personality

In [19]:
script_list = get_links(urls)
movie_list = get_titles(urls)
count, soup = get_html(script_list[1])
for character in list(count.keys()):
    dialogue = extract_dialogue(soup, [character])
    cleaned_dialogue = clean_text(dialogue, [character])
    s = retrieve_in_string(cleaned_dialogue)

In [21]:
l = []
characters_in = []
for script in script_list:
    count, soup = get_html(script)
    i = 0
    for character in list(count.keys()):
        dialogue = extract_dialogue(soup, [character])
        cleaned_dialogue = clean_text(dialogue, [character])
        s = retrieve_in_string(cleaned_dialogue)
        if len(cleaned_dialogue) > 20:
            s = retrieve_in_string(cleaned_dialogue)
            title = script[script.rfind("/") + 1 :script.find(".html")]
            l.append([title, character, s])
            i = i + 1
    characters_in.append(i)

In [22]:
personality = []
curr = 0
for i, num_actors in enumerate(characters_in):
    url = "https://www.personality-database.com/search?keyword=" + (movie_list[i]).replace(' ', '%20') + "&type=subcategories"
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/section/main/div[1]')))
    try:
        option = driver.find_element(By.CLASS_NAME, 'subcategory-card')
        movie = (option.find_element(By.CLASS_NAME, 'name')).text
        regex = generate_regex(str(movie_list[i]))
        if (re.search(regex, movie)):
            movie_url = option.get_attribute('href')
            for i, person in enumerate(l[curr:(curr + characters_in[i])]):
                l[i + curr].extend(scrape_personalities(person[1], movie_url))
        # In the case that there are results, but none are relevant movies:
        else:
            for i, person in enumerate(l[curr:(curr + characters_in[i])]):
                l[i + curr].extend(["", "", "", "", ""])
    # In the case that there are no search results for given movie query:
    except:
        for i, person in enumerate(l[curr:(curr + characters_in[i])]):
                l[i + curr].extend(["", "", "", "", ""])
        curr = curr + num_actors
        continue
    curr = curr + num_actors
driver.quit()

Found Kat Stratford
Found Patrick Verona
Found Bianca Stratford
Found Cameron James
Found Michael Eckman
Found James Cole
Found Kathryn Railly
Found Jeffrey Goines
Found David Bowman
Found Frank Poole
Found Dr. Heywood Floyd
Found Hal 9000
Found Virgil "Bud" Brigman
Found Lindsey Brigman
Found Alan "Hippy" Carnes
Found Lt. Hiram Coffey
Found Lisa "One Night" Standing
Found Ace Ventura
Found Melissa Robinson
Found Lt. Lois Einhorn / Ray Finkle
Found Arthur Dallas
Found Ellen Ripley
Found Ash
Found Joan Lambert
Found Dennis Parker
Found Arthur Dallas
Found Ellen Ripley
Found Ash
Found Joan Lambert
Found Dennis Parker
Found Margo Channing
Found Eve Harrington
Found Karen Richards
Found Bill Sampson
Found Addison DeWitt
Found Wolfgang Amadeus Mozart
Found Antonio Salieri
Found Constanze Mozart
Found Leopold Mozart
Found Kevin Myers
Found Jim Levenstein
Found Chris "Oz" Ostreicher
Found Patrick Bateman
Found Detective Donald Kimball
Found Jean
Found Courtney Rawlinson
Found Patrick Bateman


Found Louis Creed
Found Jud Crandall
Found Rachel Creed
Found Władysław Szpilman
Found Henryk Szpilman
Found Stew Smith
Found Gallagher
Found PVT Chris Taylor
Found Staff Sergeant Bob Barnes
Found Sgt. Elias Grodin
Found David
Found Jennifer
Found Betty Parker
Found Alan "Dutch" Schaefer
Found Al Dillon
Found Inigo Montoya
Found Fezzik
Found Westley
Found Buttercup
Found The Impressive Clergyman
Found Vincent Vega
Found Jules Winnfield
Found Butch Coolidge
Found Fabienne
Found Mia Wallace
Found Jake LaMotta
Found Joey LaMotta
Found Vickie
Found Chris Knight
Found Mitch Taylor
Found Kent
Found Jordan Cochran
Found Oswell E. Spencer
Found James Carter
Found Yan Naing Lee
Found Simon Templar "The Saint"
Found Ivan Tretiak
Found CPT John Miller
Found PVT Richard Reiben
Found CPL Timothy Upham
Found PVT Daniel Jackson
Found Oskar Schindler
Found Amon Goeth
Found Itzhak Stern
Found Ethan Edwards
Found Chip Sutphin
Found Misty Sutphin
Found Ellis Boyd “Red” Redding
Found Andy Dufresne
Found J

In [80]:
import pandas as pd
df = pd.DataFrame(l)
df.columns = ["Movie", "Character", "Dialogue", "Introversion/Extroversion", "Sensing/Intuition", "Thinking/Feeling", "Judging/Perceiving", "Description"]

In [42]:
df.to_csv("scripts_2.csv")

In [81]:
empty_rows = df.iloc[:, 5] == ""

# Filter out the rows that contain an empty string in the 6th column
df_filtered = df.loc[~empty_rows]
df_filtered = df_filtered[df_filtered.iloc[:, 6].notnull()]

# Take the first 100 rows of the filtered dataframe
df_first_100 = df_filtered.head(100)

In [82]:
df_filtered

,Movie,Character,Dialogue,Introversion/Extroversion,Sensing/Intuition,Thinking/Feeling,Judging/Perceiving,Description
0,10Things,KAT,"KAT STRATFORD, eighteen, pretty -- but trying ...",I,N,T,J,Kat Stratford is a fictional character who app...
1,10Things,PATRICK,Mother Goose? The class titters. Kat wears an ...,I,S,T,P,#HeathLedger as Patrick Verona
2,10Things,BIANCA,"BIANCA STRATFORD, a beautiful sophomore, stand...",E,S,F,J,
3,10Things,CAMERON,"CAMERON JAMES, a clean-cut, easy-going senior ...",I,N,F,P,#JosephGordonLevitt
4,10Things,MICHAEL,You the new guy?\nC'mon. I'm supposed to give ...,E,N,T,P,#DavidKrumholtz
6,twelve_monkeys,RAILLY,He's been tested for drugs?\nRAILLY'S POV THRO...,I,N,F,P,
7,twelve_monkeys,JEFFREY,How much you gonna pay me? Huh? I'd be doing y...,E,N,T,P,#BradPitt as Jeffrey Goines
25,2001,BOWMAN,BOWMAN INSIDE\nBOWMAN EXITS TO\nBOWMAN ENTERS ...,I,N,T,P,
26,2001,POOLE,"Right. Just a sec.\nOkay, come on down.\nPOOLE...",I,S,T,P,
27,2001,FLOYD,"FLOYD CAN BE SEEN\nFLOYD AWAKE BUT GROGGY,\nHo...",E,N,T,P,


In [83]:
print(len(df_filtered))

532
